## 1 Introduction

This tutorial introduces the basic features for simulating titratable systems via the constant pH method.
The constant pH method is a Monte-Carlo method which models reactions by randomly adding a charged particle
in the system and adding a charge of opposite sign on a neutral particle, or removing a charged particle and
neutralizing a particle of the opposite charge.

We will consider a homogeneous aqueous solution of a titratable acidic species $\mathrm{HA}$ that can dissociate as follows
$\mathrm{HA} \Leftrightarrow \mathrm{A}^- + \mathrm{H}^+$

If $N_0 = N_{\mathrm{HA}} + N_{\mathrm{A}^-}$ is the number of titratable groups in solution, then the degree of dissociation $\alpha$ can be defined:
$\alpha = \dfrac{N_{\mathrm{A}^-}}{N_0}.$

In the constant pH method, the acceptance probability for a reaction is [1]:

$ P_{\mathrm{acc}} = \operatorname{min}\left(1, \exp(\beta \Delta E_\mathrm{pot} \pm \ln_{10} (\mathrm{pH - p}K_a) ) \right)$

and the proposal probability of a reaction is $P_\text{acc}=\frac{N_\mathrm{HA}}{N_0}$ for a dissociation and $P_\text{acc}=\frac{N_\mathrm{A}}{N_0}$ for an association reaction [1]. Here $\Delta E_\text{pot}$ is the potential energy change due to the reaction, while $\text{pH - p}K_a$ is an input parameter composed by two terms, pH and $-\mathrm{p}K_a$ (the logarithm of the acid dissociation constant $K_a$).
The prefactor $\pm 1$ defines the direction of the reaction ($+1$ dissociation, $-1$ association).
When a dissociation move is attempted, a titratable molecule $\mathrm{HA}$ is given a negative charge and a counterion $\mathrm{H}^+$ is randomly placed into the simulation box, while when an association move is attempted, a $\mathrm{A}^-$ is neutralized and a random counterion $\mathrm{H}^+$ is removed from the cell.
Be aware that in the constant pH method the system is coupled to an implicit pH bath. That especially means that the number of $\mathrm{H}^+$ ions in the simulation box does not correspond to the chosen pH. This will lead to wrong screening effects when including electrostatic interactions in the system [1]. How to avoid these artifacts of the constant pH method, however, goes beyond the scope of this simple tutorial. We refer the reader to e.g. [1] for further details.

## 2 Setup
We start by creating a system instance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.ion()

import espressomd
from espressomd import code_info
from espressomd import analyze
from espressomd import integrate
from espressomd import reaction_ensemble

# System parameters
#############################################################
box_l = 35

# Integration parameters
#############################################################
system = espressomd.System(box_l=[box_l] * 3)
system.set_random_state_PRNG()
np.random.seed(seed=10)

system.time_step = 0.02
system.cell_system.skin = 0.4

Next we define some important constants and add particles to the system:

In [ ]:
type_HA = 0
type_A = 1
type_H = 2

N0 = 50  # number of titratable units
K = 1e-4
pK=-np.log10(K)

for i in range(N0):
    system.part.add(id=i, pos=np.random.random(3) * system.box_l, type=type_A)
for i in range(N0, 2 * N0):
    system.part.add(id=i, pos=np.random.random(3) * system.box_l, type=type_H)

The next step is to define the reaction system and the seed of the pseudo-random number generator which is used for the Monte-Carlo steps.
Please note that the order in which species are written in the reactants and products lists is very important because, when a reaction is performed, the first species in the reactants list is replaced by the first species in the product lists, the second reactant species is replaced by the second product species, and so on. Moreover, if the reactant list has more species than the products list, reactant molecules in excess are deleted from the system, while if the products list has more species than the reactants list, product molecules in excess are created and randomly placed inside the simulation box. For example, reversing the order of products in our reaction (i.e. from  `product_types=[type_H, type_A]` to `product_types=[type_A, type_H]`), a neutral monomer would be positively charged and a negatively charged monovalent counterion would be randomly placed inside the cell. 

In [ ]:
RE = reaction_ensemble.ConstantpHEnsemble(
        temperature=1, exclusion_radius=0, seed=77)

RE.add_reaction(gamma=K, reactant_types=[type_HA], reactant_coefficients=[1],
                product_types=[type_A, type_H], product_coefficients=[1, 1],
                default_charges={type_HA: 0, type_A: -1, type_H: +1})
print(RE.get_status())

Next we perform simulations at different pH values. The system must be equilibrated at each pH before taking samples.

In [ ]:
num_samples=30

pHs=np.linspace(1, 8, num=15)
degrees_of_dissociation = []
std_dev_degree_of_dissociation = []
histograms=[]
histogram_edges=range(0,N0+1)

for pH in pHs:
    print("pH is {:.1f}".format(pH))
    RE.constant_pH = pH
    RE.reaction(4 * N0) # warmup
    numAs = []
    for i in range(num_samples):
        RE.reaction(N0+1)
        numAs.append(system.number_of_particles(type=type_A))
    degrees_of_dissociation.append(np.mean(numAs) / N0)
    std_dev_degree_of_dissociation.append(np.std(numAs, ddof=1) / N0)
    print("occured particle numbers of type A ", sorted(set(numAs)))
    histogram=np.bincount(np.array(numAs).astype(int), minlength=N0+1)
    histograms.append(histogram/float(len(numAs)))

## 3 Results

Finally we plot our results and compare it to the well-known result for an ideal reacting system. From the definition of the acid dissociation constant $K_a$:

\begin{equation}
K_a = \frac{[\mathrm{H}^+][\mathrm{A}^-]}{[\mathrm{HA}]} \frac{\gamma_{\mathrm{H}^+}\gamma_{\mathrm{A}^-}}{\gamma_{\mathrm{HA}}},
\end{equation}

with $[\mathrm{H}^+]$ and $\gamma_{\mathrm{H}^+}$ the concentration and activity coefficient of species $\mathrm{H}^+$, respectively.
For highly diluted system, the activity coefficients becomes unity and $[\mathrm{H}^+] = 10^{-\mathrm{pH}}$.

Dividing both the numerator and denominator of alpha by the simulation box volume $V$ allows us to express $\alpha$ as a ratio of concentrations:

\begin{equation}
\alpha = \frac{N_{\mathrm{A}^-}}{N_0} \cdot \frac{V}{V} = \frac{N_{\mathrm{A}^-}}{N_{\mathrm{HA}} + N_{\mathrm{A}^-}} \cdot \frac{V}{V} = \frac{[\mathrm{A}^-]}{[\mathrm{HA}]+[\mathrm{A}^-]}.
\end{equation}

Multiplying both the numerator and denominator by $\dfrac{[\mathrm{H}^+]}{[\mathrm{HA}]}$ leads to:

\begin{equation}
\alpha = \frac{\frac{[\mathrm{H}^+][\mathrm{A}^-]}{[\mathrm{HA}]}}{\frac{[\mathrm{H}^+][\mathrm{HA}]}{[\mathrm{HA}]}+\frac{[\mathrm{H}^+][\mathrm{A}^-]}{[\mathrm{HA}]}} = \frac{K_a}{[\mathrm{H}^+] + K_a}.
\end{equation}

Since $K_a = 10^{-\mathrm{p}K_a}$ we can express $\alpha$ as a simple function of the pH and $\mathrm{p}K_a$:

\begin{equation}
\alpha = \frac{1}{\frac{[\mathrm{H}^+]}{K_a} + 1} = \frac{1}{\frac{10^{-\mathrm{pH}}}{10^{-\mathrm{p}K_a}} + 1} = \frac{1}{10^{\mathrm{p}K_a-\mathrm{pH}} + 1}.
\end{equation}

We will plot the results of the simulation against the theoretical curve. The numerical errors in the simulation can be reduced by increasing `num_samples` taken at each pH. To quantify the simulation uncertainty, we will add error bars to the calculated data points. For simplicity, we will estimate the error of the degree of dissociation using the standard error of the mean (SE) for $N$ _uncorrelated_ samples:

\begin{equation}
    \mathrm{SE}      = \frac{\sigma_N}{\sqrt{N}},
\end{equation}

where $\sigma_N$ is the sample standard deviation, defined as:

\begin{equation}
    \sigma_N^2 = \left\langle \alpha^2 - \langle \alpha\rangle_N^2 \right\rangle_N
\end{equation}

where $\langle\alpha\rangle_N$ is the sample mean. The probability of finding the true mean $\mu$ (estimated by $\langle\alpha\rangle_N$) within two bounds at 95% is given by:

\begin{equation}
    \operatorname{Pr}\left( \langle\alpha\rangle_N -z\cdot\mathrm{SE} \leq \mu \leq \langle\alpha\rangle_N -z\cdot\mathrm{SE}\right) = 0.95,
\end{equation}

where $z \approx 1.96$ is the two-sided confidence level for the normal distribution at 95%.

Because our sample size $n$ is small, we will use the Bessel-corrected sample standard deviation $s$:

\begin{equation}
    s = \sqrt{\frac{n}{n-1}\left\langle \alpha^2 - \langle \alpha\rangle_n^2 \right\rangle_n}
\end{equation}

The standard error of the mean (SE) is estimated by $s_{\bar\alpha}$:

\begin{equation}
    s_{\bar\alpha} = \frac{s}{\sqrt{n}}
\end{equation}

The confidence interval is defined as:

\begin{equation}
    \mathrm{CI}_{95\%} = \langle\alpha\rangle_n \pm z \cdot s_{\bar\alpha}
\end{equation}

In [ ]:
import scipy.stats
def ideal_degree_of_dissociation(pH, pK):
    return 1. / (1 + 10**(pK - pH))

z = scipy.stats.norm.interval(0.95, loc=0)[1]
ci95 = z * np.array(std_dev_degree_of_dissociation) / np.sqrt(num_samples)
pK = -np.log10(K)
plt.figure(figsize=(10, 6), dpi=80)
plt.errorbar(pHs - pK, degrees_of_dissociation, ci95, label=r"simulation")
pHs2=np.linspace(0,8,num=50)
plt.plot(pHs2 - pK, ideal_degree_of_dissociation(pHs2, pK), label=r"ideal")
plt.xlabel('$pH-pK$', fontsize=20)
plt.ylabel(r'$\alpha$', fontsize=20)
plt.legend(fontsize=20)
plt.show()

In addition we compare the relative frequencies which are observed for the number of particles of type A to the probabilities which are expected in the constant pH ensemble for a non interacting system [1]: $p(N_A)=\frac{{N_0 \choose N_A} 10^{(pH-pKa)N_A}}{\sum_{N_A=0}^{N_0} {N_0 \choose N_A} 10^{(pH-pKa)N_A}}$

In [ ]:
import scipy.special
def constant_pH_numA_pmf(numA,pH, pK, N0):
        all_NAs=range(N0+1)
        return scipy.special.binom(N0, numA)*10.0**((pH-pK)*numA)/sum(scipy.special.binom(N0, i)*10**((pH-pK)*i) for i in all_NAs)

NAs=range(N0+1)

from IPython import display
import time

for index_pH in range(pHs.shape[0]):
    fig, ax = plt.subplots(1, 1)
    pH=pHs[index_pH]
    ax.plot(NAs, constant_pH_numA_pmf(NAs, pH, pK, N0), 'ro', ms=12, mec='r', label="Theory pH "+str(pH))
    ax.vlines(NAs, 0, constant_pH_numA_pmf(NAs, pH, pK, N0), colors='r', lw=4)
    ax.plot(histogram_edges, histograms[index_pH], 'bo', ms=12, mec='b', label="Simulation pH "+str(pH))
    ax.vlines(histogram_edges, 0, histograms[index_pH], colors='blue', lw=1)
    ax.legend()
    display.clear_output(wait=True)
    display.display(plt.gcf())
    time.sleep(2.0)

## References

[1] Landsgesell, Jonas, Christian Holm, and Jens Smiatek. Simulation of weak polyelectrolytes: a comparison between the constant pH and the reaction ensemble method. The European Physical Journal Special Topics 226.4 (2017): 725-736.